In [1]:
from agentic_blocks.utils.tools_utils import create_tool_registry, execute_and_add_tool_responses
from agentic_blocks.utils.tools_utils import execute_pending_tool_calls
from agentic_blocks.utils.tools_utils import langchain_tool_to_openai_format
from agentic_blocks import call_llm, Messages
from deep_research.tools import internet_search, write_todos, think_tool
from IPython.display import display, Markdown

In [2]:
tools = [internet_search, write_todos, think_tool]

tool_registry = create_tool_registry(tools)

#langchain_tool_to_openai_format(think_tool)

In [3]:
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

First use the write_todos tool to create a todo list.

When you have a list of todos, use the internet_search tool to conduct deep research. It will respond to your questions/topics with a detailed answer.

When you think you enough information to write a final report provide the final report.

Here are instructions for writing the final report:

<report_instructions>

CRITICAL: If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!

Note: Write the report in English.

Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. References relevant sources using [Title](URL) format
4. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
5. Includes a "Sources" section at the end with all referenced links

You can structure your report in a number of different ways. Here are some examples:

To answer a question that asks you to compare two things, you might structure your report like this:
1/ intro
2/ overview of topic A
3/ overview of topic B
4/ comparison between A and B
5/ conclusion

To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
1/ list of things or table of things
Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
1/ item 1
2/ item 2
3/ item 3

To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
1/ overview of topic
2/ concept 1
3/ concept 2
4/ concept 3
5/ conclusion

If you think you can answer the question with a single section, you can do that too!
1/ answer

REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
Make sure that your sections are cohesive, and make sense for the reader.

For each section of the report, do the following:
- Use simple, clear language
- Use ## for section title (Markdown format) for each section of the report
- Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
- Do not say what you are doing in the report. Just write the report without any commentary from yourself.
- Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
- Use bullet points to list out information when appropriate, but by default, write in paragraph form.

REMEMBER:
The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
Make sure the final answer report is in English.

Format the report in clear markdown with proper structure and include source references where appropriate.

<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- IMPORTANT: Number sources sequentially without gaps (1,2,3,4...) in the final list regardless of which sources you choose
- Each source should be a separate line item in a list, so that in markdown it is rendered as a list.
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
- Citations are extremely important. Make sure to include these, and pay a lot of attention to getting these right. Users will often use these citations to look into more information.
</Citation Rules>
</report_instructions>

You have access to these tools:

## `write_todos`

Use this to create and manage a structured task list. Pass an array of todo objects, each with:
- `content`: A string describing the task (required)
- `status`: One of "pending", "in_progress", or "completed" (optional, defaults to "pending")

Example: write_todos(todos=[{"content": "Research top restaurants", "status": "pending"}, {"content": "Write report", "status": "pending"}])

## `internet_search`

Use this to run an internet search for a given query. You can specify the number of results, the topic, and whether raw content should be included.

## `think_tool`

Use this to think about the research you have done so far. This will help you make decisions about what to do next.
"""

In [6]:
messages = Messages(
    system_prompt=research_instructions,
    user_prompt="Make a thorough comparison of Langgraph and Pydantic AI with pros and cons"
)

model = "qwen/qwen3-235b-a22b-2507"
#model = "openai/gpt-oss-120b"

In [7]:
has_answer = False

while not has_answer:
    response = call_llm(model=model, messages=messages, tools=tools)
    messages.add_response_message(response)
    #print(messages.get_messages()[-1])
    
    if messages.has_pending_tool_calls():
        for tool_call in messages.get_pending_tool_calls():
            print(f"## Tool call: {tool_call['tool_name']}: {tool_call['arguments']}")
        #print(messages.get_pending_tool_calls())
        tool_responses = execute_pending_tool_calls(messages, tool_registry)
        messages.add_tool_responses(tool_responses)
    else:
        has_answer = True
    
    

## Tool call: write_todos: {'todos': [{'content': 'Research Langgraph: overview, features, use cases, and user feedback', 'status': 'pending'}, {'content': 'Research Pydantic AI: overview, features, use cases, and user feedback', 'status': 'pending'}, {'content': 'Compare Langgraph and Pydantic AI on key criteria: architecture, use cases, ease of use, integration, performance, community, and developer experience', 'status': 'pending'}, {'content': 'Compile pros and cons for each framework based on research findings', 'status': 'pending'}, {'content': 'Write final report in English comparing Langgraph and Pydantic AI with pros and cons', 'status': 'pending'}]}
-------------- Writing todos --------------
- [pending] Research Langgraph: overview, features, use cases, and user feedback
- [pending] Research Pydantic AI: overview, features, use cases, and user feedback
- [pending] Compare Langgraph and Pydantic AI on key criteria: architecture, use cases, ease of use, integration, performanc

In [8]:
display(Markdown(response.content))

# Comparison of LangGraph and Pydantic AI: A Comprehensive Analysis

## Overview of LangGraph

LangGraph is a stateful orchestration framework developed by the LangChain team, specifically designed for building complex, multi-agent applications powered by large language models. As part of the LangChain ecosystem, LangGraph uses graph-based architectures to model and manage intricate relationships between components in AI agent workflows. The framework represents processes as nodes and edges in a computational graph, where nodes are individual agents or components and edges represent the flow of data and control between them.

LangGraph is particularly effective for applications requiring persistent state across multiple interactions, cyclical processing patterns, and complex decision-making workflows. It provides reliability and controllability through features like moderation checks, human-in-the-loop approvals, and persistence of context for long-running workflows. The framework offers first-class streaming support, enabling token-by-token output streaming and visibility into intermediate steps, which provides transparency into agent reasoning during execution. Developed to move beyond simple linear pipelines, LangGraph supports loops, conditional branching, and complex agent interactions, making it suitable for sophisticated AI systems that require coordination between multiple specialized agents [What is LangGraph? - IBM](https://www.ibm.com/think/topics/langgraph).

## Overview of Pydantic AI

Pydantic AI is a Python agent framework that extends the popular Pydantic library, which is widely known for data validation and settings management in Python applications. Rather than creating a standalone platform, Pydantic AI acts as a bridge between developers and language models, providing tools to create agents that execute specific tasks based on system prompts, functions, and structured outputs. The framework leverages Python's type hints to enforce data types and constraints, ensuring data integrity throughout AI workflows.

Pydantic AI is designed to make it less painful to build production-grade applications with generative AI by focusing on structured data models, testability, and integration with existing development workflows. Developers define data models with type annotations, which Pydantic AI uses to validate input and manage agent interactions. A key feature is its evaluation framework (pydantic_evals) which helps address the challenge of testing and evaluating AI agents through systematic tools and methodologies. The integration with Pydantic Logfire provides built-in observability, allowing real-time tracing of prompts, costs, and agent behaviors. Pydantic AI emphasizes a Pythonic approach with type-safe outputs and structured prompts, making it intuitive for developers already familiar with Python's typing system [Pydantic: Fast Data Validation & Settings Library Guide](https://justcall.io/ai-agent-directory/pydantic/).

## Architectural Comparison

### LangGraph Architecture

LangGraph employs a graph-based architectural model where workflows are represented as directed graphs with nodes and edges. This architectural approach allows for exceptional flexibility in workflow design, supporting non-linear execution patterns including loops, conditional branching, and parallel processing. The framework's statefulness ensures that context is maintained throughout the workflow execution, which is critical for complex, long-running processes that may span multiple interactions.

The graph structure in LangGraph provides clear control flow, making it easier to design, implement, and reason about complex multi-agent workflows. Nodes in the graph can represent various components such as individual AI agents, function calls, or human interaction points, while edges define the sequence and conditions for transitioning between nodes. This architecture enables sophisticated coordination between agents and supports use cases requiring fine-grained control over execution flow. Built-in capabilities like persistence, streaming, and human-in-the-loop support make LangGraph suitable for real-world, error-tolerant systems that require enterprise-grade reliability [What is LangGraph? Key Concepts](https://www.designveloper.com/blog/what-is-langgraph/).

### Pydantic AI Architecture

Pydantic AI follows a more minimalist and Python-centric architectural approach, building upon the existing Pydantic library's robust data validation capabilities. Instead of a visual graph-based model, Pydantic AI focuses on code-driven agent definitions with strong typing and data structure enforcement. The framework integrates seamlessly with existing Python applications and development workflows, allowing developers to define agents through straightforward Python classes and functions.

The architecture emphasizes dependency injection, structured outputs, and data integrity validation at every step of the workflow. By leveraging Python's type system, Pydantic AI ensures that inputs and outputs conform to predefined data models, reducing runtime errors and improving reliability. The framework's design philosophy centers on keeping code clean and maintainable, with a focus on rapid prototyping capabilities. Unlike LangGraph's complex orchestration model, Pydantic AI operates more as a lightweight shim that connects Pydantic's validation strengths with LLM interactions, resulting in a simpler but highly effective approach for many use cases [Pydantic-AI: A Powerful Alternative](https://tech.appunite.com/posts/understanding-pydantic-ai-a-powerful-alternative-to-lang-chain-and-llama-index).

## Use Case Analysis

### LangGraph Use Cases

LangGraph excels in complex, enterprise-grade applications that require sophisticated orchestration, multi-agent coordination, and long-term state management. Its graph-based architecture makes it particularly suitable for scenarios involving iterative problem-solving, where agents may need to loop back to previous steps based on new information or outcomes. Examples include customer support chatbots that need to maintain conversation state across multiple interactions, complex decision support systems that coordinate between specialized agents (research, analysis, recommendation), and workflow automation systems that require human-in-the-loop approvals at various decision points.

The framework is trusted by companies like Uber and Replit for applications requiring fine-grained control and scalability. LangGraph is ideal for use cases where observability is critical, such as financial systems, healthcare applications, or any domain where audit trails and transparency into agent reasoning are required. Its ability to handle long-running processes with persistent state makes it suitable for applications like project management assistants, research synthesis tools, or any system where context must be maintained over extended periods [Pydantic AI vs LangGraph: Which Agent Framework Wins](https://www.youtube.com/watch?v=aNrd24DL6fs).

### Pydantic AI Use Cases

Pydantic AI shines in applications where data integrity, rapid prototyping, and clean code organization are priorities. Its strength lies in building production-grade applications that require strict validation of inputs and outputs, making it ideal for API integrations, data processing pipelines, and applications where reliability and consistency are paramount. The framework is particularly effective for use cases involving structured data extraction, form processing, or any scenario where LLM outputs need to conform to specific data schemas.

Common applications include customer data processing systems that extract information from unstructured text into validated data models, financial reporting tools that ensure output formats are consistent, and any workflow where reducing runtime errors through type validation would provide significant value. Pydantic AI is also well-suited for development teams that already use Pydantic in their stack and want to extend those benefits to their AI applications. The framework's evaluation tools make it excellent for applications where testing and validation of agent performance are critical requirements [Understanding Pydantic-AI](https://tech.appunite.com/posts/understanding-pydantic-ai-a-powerful-alternative-to-lang-chain-and-llama-index).

## Developer Experience and Learning Curve

### LangGraph Developer Experience

LangGraph presents a steeper learning curve due to its graph-based conceptual model and the need to understand graph theory concepts for effective implementation. Developers must learn to design workflows as directed graphs, manage state transitions between nodes, and implement complex branching logic. The framework requires understanding of concepts like state persistence, edge conditions, and node coordination, which adds cognitive overhead compared to simpler linear approaches.

However, for complex applications, this investment in learning pays dividends through superior control and orchestration capabilities. The framework provides comprehensive tools for debugging and monitoring agent workflows, with transparency into the agent's state at each step. For developers already familiar with the LangChain ecosystem, the transition to LangGraph is smoother, as it shares many conceptual models and patterns. The visual representation of workflows can aid in understanding and troubleshooting complex agent interactions, though it requires mastering the framework's specific syntax and patterns [Pydantic AI vs LangGraph: The Ultimate Developer's Guide](https://atalupadhyay.wordpress.com/2025/07/10/pydantic-ai-vs-langgraph-the-ultimate-developers-guide/).

### Pydantic AI Developer Experience

Pydantic AI offers a more intuitive developer experience, particularly for Python developers already familiar with type hints and the Pydantic library. The learning curve is significantly lower, as developers can leverage their existing knowledge of Python's typing system and Pydantic's validation patterns. The framework "clicks right away" for developers who understand Python typing, allowing rapid implementation without needing to learn new conceptual models [Comparing Pydantic AI with Langgraph](https://community.latenode.com/t/comparing-pydantic-ai-with-langgraph-for-agent-development/31002).

The code-centric approach, with agents defined through regular Python classes and functions, feels more natural to experienced Python developers. This familiarity enables faster development cycles and easier code maintenance. Pydantic AI excels at keeping code clean and organized, which contributes to better long-term maintainability. The integration with existing development tools and workflows, including GitHub, makes it easier to incorporate into CI/CD pipelines and version control systems. For teams prioritizing developer productivity and rapid prototyping, Pydantic AI provides a more seamless experience.

## Pros and Cons Comparison

### LangGraph Pros

- **Superior Orchestration Capabilities**: LangGraph's graph-based architecture enables complex workflows with loops, conditional branching, and parallel processing that are difficult to implement with linear frameworks [What is LangGraph? - IBM](https://www.ibm.com/think/topics/langgraph)
- **Advanced State Management**: Excellent state persistence for long-running processes, maintaining context across multiple interactions and workflow cycles
- **Multi-Agent Coordination**: Specifically designed for coordinating multiple specialized agents, making it ideal for complex problem-solving scenarios
- **Enterprise-Grade Features**: Offers robust observability, moderation checks, and human-in-the-loop capabilities required for production systems
- **Scalability**: Proven at enterprise scale with companies like Uber and Replit using it for mission-critical applications
- **Transparency**: Provides full visibility into agent state and reasoning processes, which is essential for debugging and compliance
- **Ecosystem Integration**: Benefits from the extensive LangChain ecosystem of tools, integrations, and community support

### LangGraph Cons

- **Steeper Learning Curve**: Requires understanding of graph theory concepts and complex state management patterns, creating a barrier to entry
- **Overhead for Simple Applications**: May introduce unnecessary complexity for straightforward AI workflows that don't require multi-agent coordination
- **Performance Considerations**: The orchestration layer can add computational overhead compared to simpler frameworks
- **Resource Intensive**: Managing state and complex workflows requires more system resources and careful optimization
- **Development Speed**: The complexity of the framework can slow down initial development compared to more streamlined approaches

### Pydantic AI Pros

- **Type Safety and Data Integrity**: Leverages Pydantic's robust validation to ensure data consistency and reduce runtime errors
- **Faster Development**: Enables rapid prototyping and quicker iteration cycles due to familiar Python patterns
- **Cleaner Code**: Produces more maintainable, organized code that follows Python best practices
- **Lower Learning Curve**: Intuitive for developers already familiar with Python typing and the Pydantic library
- **Seamless Integration**: Integrates smoothly with existing Python applications and development workflows
- **Built-in Testing Framework**: Comprehensive evaluation tools (pydantic_evals) for systematically testing and validating agents
- **Observability**: Integration with Pydantic Logfire provides real-time tracing of prompts, costs, and agent behaviors
- **Production Focus**: Specifically designed to make building production-grade applications less painful [GitHub - pydantic/pydantic-ai](https://github.com/pydantic/pydantic-ai)

### Pydantic AI Cons

- **Limited Orchestration**: Less suited for complex workflows with loops, conditional branching, and multi-agent coordination
- **Simpler State Management**: State persistence capabilities are not as robust as LangGraph for long-running, complex processes
- **Narrower Scope**: Focuses primarily on agent creation and data validation rather than comprehensive workflow orchestration
- **Emerging Ecosystem**: As a newer framework, the ecosystem of tools and integrations is not as extensive as LangChain/LangGraph
- **Less Visual Workflow Design**: Code-driven approach doesn't provide the same visual representation of workflows as graph-based models

## Conclusion and Recommendation

The choice between LangGraph and Pydantic AI ultimately depends on the specific requirements of the project and the development team's expertise and priorities. LangGraph is the superior choice for complex, enterprise-grade applications that require sophisticated orchestration, multi-agent coordination, and long-term state management. Its graph-based architecture provides unparalleled flexibility for designing workflows with loops, conditional branching, and parallel processing, making it ideal for applications like customer support systems, decision support tools, and complex automation workflows that need fine-grained control and observability.

Pydantic AI, on the other hand, excels at rapid prototyping, data validation, and building production-grade applications with clean, maintainable code. It's particularly well-suited for teams already using Pydantic in their stack and developers familiar with Python's type system. The framework's strength in ensuring data integrity and its lower learning curve make it an excellent choice for applications requiring structured outputs, API integrations, and reliable data processing.

For projects requiring complex workflows with serious observability and branching logic, LangGraph remains the preferred solution. However, for rapid development, cleaner code, and applications where type safety and data validation are paramount, Pydantic AI offers significant advantages. Developers should consider LangGraph for enterprise-level applications with complex agent interactions, while Pydantic AI is better suited for projects prioritizing development speed, code quality, and data integrity in production environments [Comparing Pydantic AI with Langgraph](https://community.latenode.com/t/comparing-pydantic-ai-with-langgraph-for-agent-development/31002).

### Sources
[1] What is LangGraph? - IBM: https://www.ibm.com/think/topics/langgraph  
[2] What is LangGraph? Key Concepts: https://www.designveloper.com/blog/what-is-langgraph/  
[3] Pydantic: Fast Data Validation & Settings Library Guide: https://justcall.io/ai-agent-directory/pydantic/  
[4] Pydantic-AI: A Powerful Alternative: https://tech.appunite.com/posts/understanding-pydantic-ai-a-powerful-alternative-to-lang-chain-and-llama-index  
[5] Pydantic AI vs LangGraph: Which Agent Framework Wins: https://www.youtube.com/watch?v=aNrd24DL6fs  
[6] Pydantic AI vs LangGraph: The Ultimate Developer's Guide: https://atalupadhyay.wordpress.com/2025/07/10/pydantic-ai-vs-langgraph-the-ultimate-developers-guide/  
[7] Comparing Pydantic AI with Langgraph: https://community.latenode.com/t/comparing-pydantic-ai-with-langgraph-for-agent-development/31002  
[8] GitHub - pydantic/pydantic-ai: https://github.com/pydantic/pydantic-ai